In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.callbacks import EarlyStopping
from keras.applications.inception_v3 import InceptionV3, preprocess_input

In [ ]:
size = [224, 224]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_data = '/content/drive/MyDrive/HDA2/Covid19-dataset/train'
test_data = '/content/drive/MyDrive/HDA2/Covid19-dataset/test'

In [ ]:
inception = InceptionV3(input_shape=size + [3], weights='imagenet', include_top=False)


In [ ]:
for layer in inception.layers:
    layer.trainable = False

In [ ]:
data = glob('/content/drive/MyDrive/HDA2/Covid19-dataset/train/*')

In [ ]:
x = Flatten()(inception.output)
prediction = Dense(len(data), activation='softmax')(x)
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_img = ImageDataGenerator(rescale=1./255, shear_range=0.25, zoom_range=0.3)
test_img = ImageDataGenerator(rescale=1./255)
training_set = train_img.flow_from_directory('/content/drive/MyDrive/HDA2/Covid19-dataset/train', target_size=size)
testing_set = test_img.flow_from_directory('/content/drive/MyDrive/HDA2/Covid19-dataset/test',
                                          target_size=(224, 224),
                                          batch_size=8,
                                          class_mode='categorical')

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

r = model.fit(
  training_set,
  validation_data=testing_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(testing_set),
  callbacks=[early_stop]
)

Epoch 1/50
8/8 [==============================] - 82s 10s/step - loss: 14.4738 - accuracy: 0.4542 - val_loss: 6.8150 - val_accuracy: 0.4545
Epoch 2/50
8/8 [==============================] - 47s 6s/step - loss: 4.3063 - accuracy: 0.6335 - val_loss: 3.2052 - val_accuracy: 0.6364
Epoch 3/50
8/8 [==============================] - 47s 6s/step - loss: 1.5958 - accuracy: 0.8207 - val_loss: 1.7604 - val_accuracy: 0.7727
Epoch 4/50
8/8 [==============================] - 47s 6s/step - loss: 1.0881 - accuracy: 0.8845 - val_loss: 2.6282 - val_accuracy: 0.7576
Epoch 5/50
8/8 [==============================] - 69s 9s/step - loss: 0.9006 - accuracy: 0.8924 - val_loss: 1.9677 - val_accuracy: 0.8030
Epoch 6/50
8/8 [==============================] - 50s 6s/step - loss: 0.6830 - accuracy: 0.9323 - val_loss: 0.8300 - val_accuracy: 0.9242
Epoch 7/50
8/8 [==============================] - 59s 8s/step - loss: 0.6904 - accuracy: 0.9283 - val_loss: 1.4349 - val_accuracy: 0.8333
Epoch 8/50
8/8 [================

In [ ]:
model.save('Lung_disease_pred_inception.h5')

In [ ]:
from keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np


model_inception = load_model('/content/drive/MyDrive/HDA2/Lung_disease_pred_inception.h5')

In [ ]:

y_pred_inception = model_inception.predict(testing_set)


y_pred_classes_inception = np.argmax(y_pred_inception, axis=1)
y_true_classes = testing_set.classes

accuracy_inception = accuracy_score(y_true_classes, y_pred_classes_inception)
print("Accuracy (InceptionV3):", accuracy_inception)


print("Classification Report (InceptionV3):")
print(classification_report(y_true_classes, y_pred_classes_inception))


conf_matrix_inception = confusion_matrix(y_true_classes, y_pred_classes_inception)
print("Confusion Matrix (InceptionV3):")
print(conf_matrix_inception)

9/9 [==============================] - 9s 813ms/step
Accuracy (InceptionV3): 0.36363636363636365
Classification Report (InceptionV3):
              precision    recall  f1-score   support

           0       0.42      0.38      0.40        26
           1       0.33      0.30      0.32        20
           2       0.33      0.40      0.36        20

    accuracy                           0.36        66
   macro avg       0.36      0.36      0.36        66
weighted avg       0.37      0.36      0.36        66

Confusion Matrix (InceptionV3):
[[10  7  9]
 [ 7  6  7]
 [ 7  5  8]]


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay

auroc_per_class = [roc_auc_score(y_true_classes == i, y_pred_inception[:, i]) for i in range(len(data))]


for i, auroc in enumerate(auroc_per_class):
    print(f"Class {i} AUROC (InceptionV3): {auroc}")



Class 0 AUROC (InceptionV3): 0.5052884615384616
Class 1 AUROC (InceptionV3): 0.5961956521739131
Class 2 AUROC (InceptionV3): 0.5266304347826087


In [ ]:
from PIL import Image
image = '/content/drive/MyDrive/HDA2/Covid19-dataset/train/Covid/015.jpg'
image1 = Image.open(image)

In [ ]:
new_size = (224,224)


resized_image = image1.resize(new_size)


resized_image.save('output_image.jpg')

In [ ]:
resized_image = np.array(resized_image)

In [ ]:
x = np.expand_dims(resized_image,axis=0)
x = preprocess_input(x)

In [ ]:
a = model.predict(x)

1/1 [==============================] - 0s 171ms/step


In [ ]:
new_size = (224,224)


resized_image = image1.resize(new_size)


resized_image.save('output_image.jpg')

In [ ]:
resized_image = np.array(resized_image)
x = np.expand_dims(resized_image,axis=0)
x = preprocess_input(x)

In [ ]:
x.shape

(1, 224, 224, 3)

In [ ]:
np.argmax(model.predict(x))

1/1 [==============================] - 0s 312ms/step


1

In [ ]:
from PIL import Image
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import numpy as np

img_path = '/content/drive/MyDrive/HDA2/Covid19-dataset/test/Normal/0103.jpeg'



img = Image.open(img_path)
img = img.resize((224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)


In [ ]:
predictions = model.predict(img_array)


print(predictions)


predicted_class_index = np.argmax(predictions[0])
print("Predicted Class Index:", predicted_class_index)



class_labels = ['class_0', 'class_1', 'class_2']  # Replace with your actual class labels
predicted_class_label = class_labels[predicted_class_index]
print("Predicted Class Label:", predicted_class_label)

1/1 [==============================] - 0s 161ms/step
[[1.5023556e-18 1.0000000e+00 2.2782393e-14]]
Predicted Class Index: 1
Predicted Class Label: class_1
